In [346]:
import pandas as pd

In [347]:
""" Get stuff out of Netfile v2 API
"""
from pprint import PrettyPrinter
from pathlib import Path
import os
import requests

BASE_URL = 'https://netfile.com/api/campaign'
CONTRIBUTION_FORM = 'F460A'
EXPENDITURE_FORM = 'F460E'

PARAMS = { 'aid': 'COAK' }

def get_auth_from_env_file(filename: str='.env'):
    """ Split .env file on newline and look for API_KEY and API_SECRET
        Return their values as a tuple
    """
    env_file=Path(filename)
    auth_keys = [ 'API_KEY', 'API_SECRET' ]
    if env_file.exists():
        auth = tuple( v for _, v in sorted([
            ln.split('=') for ln in
            env_file.read_text(encoding='utf8').strip().split('\n')
            if ln.startswith(auth_keys[0]) or ln.startswith(auth_keys[1])
        ], key=lambda ln: auth_keys.index(ln[0])))
    else:
        auth=tuple(os.environ[key] for key in auth_keys)
            
    return auth

AUTH=get_auth_from_env_file()

pp = PrettyPrinter()

def get_filing(offset=0):
    """ Get a filing
    """
    url = f'{BASE_URL}/filing/v101/filings'

    params = { **PARAMS }
    if offset > 0:
        params['offset'] = offset

    res = requests.get(url, params=params, auth=AUTH)
    if res.status_code == 500:
        print('ping')
        return get_filing(offset=0)
    else:
        print(res)
        body = res.json()
        results = body.pop('results')
        return results, body
def get_form(form,offset=0):
    """ Get a filing
    """
    url = f'{BASE_URL}/filing/v101/filings?Limit=100000&SpecificationForm={form}'

    params = { **PARAMS }
    if offset > 0:
        params['offset'] = offset

    res = requests.get(url, params=params, auth=AUTH)
    if res.status_code == 500:
        return get_form(form,offset=0)
    else:
        body = res.json()
        results = body.pop('results')

        return results, body
def get_filer(filer_nid):
    """ Get one filer
    """
    url = f'{BASE_URL}/filer/v101/filers?'

    res = requests.get(url, params={ **PARAMS, 'filerNid': filer_nid }, auth=AUTH)
    if res.status_code == 500:
        return get_filer(filer_nid)
    else:
        body = res.json()

        return body['results']
def list_filers():
    """ Get all the elections
    """
    url = f'{BASE_URL}/filer/v101/filers?Limit=100000'

    res = requests.get(url, params=PARAMS, auth=AUTH)
    if res.status_code == 500:
        print('ping')
        return list_filers()
    else:
        body = res.json()
        return body['results']
def list_elections_influences(id):
    """ Get all the elections
    """
    url = f'{BASE_URL}/election/v101/election-influences?Limit=100000&ElectionNid={id}'

    res = requests.get(url, params=PARAMS, auth=AUTH)
    if res.status_code == 500:
        return list_elections_influences(id)
    else:
        body = res.json()
        return body['results']

def list_elections():
    """ Get all the elections
    """
    url = f'{BASE_URL}/election/v101/elections?Limit=100000'

    res = requests.get(url, params=PARAMS, auth=AUTH)
    if res.status_code == 500:
        return list_elections()
    else:
        body = res.json()

        return body['results']
def export_transactions(id,offset=0):
    """ Get a filing
    """
    url = f'{BASE_URL}/filing/v101/filings/{id}'

    params = { **PARAMS }
    if offset > 0:
        params['offset'] = offset

    res = requests.get(url, params=params, auth=AUTH)
    if res.status_code == 500:
        return export_transactions(id,offset=0)
    else:
        body = res.json()

        return body

In [348]:
filing=get_filing()
# filing

<Response [200]>


In [349]:
filers_response=list_filers()
filers_response

[{'filerNid': '211219898',
  'visibilityLevel': 'Staff',
  'filerName': 'Bartsch, Justin',
  'candidateName': None,
  'aid': 'COAK',
  'adsid': 163887,
  'isTerminated': False,
  'committeeTypes': ['Conflict of Interest'],
  'nameHistory': [],
  'addressList': [{'addressTypes': 'Disclosure, Mailing',
    'line1': '7101 Edgewater Drive',
    'line2': '',
    'city': 'Oakland',
    'state': 'CA',
    'zip': '94621'}],
  'phoneList': [{'phoneType': 'Other', 'number': '5103337447'}],
  'emailList': [{'address': 'jbartsch@oaklandca.gov',
    'emailType': 'Notification'},
   {'address': 'jbartsch@oaklandca.gov', 'emailType': 'Login'}],
  'statusItemList': [{'status': 'ACTIVE',
    'date': '2024-03-28T07:00:00+00:00'}],
  'registrations': {},
  'officers': [],
  'electionInfluences': []},
 {'filerNid': '211217137',
  'visibilityLevel': 'Staff',
  'filerName': 'Hoffman, Kelly',
  'candidateName': None,
  'aid': 'COAK',
  'adsid': 163887,
  'isTerminated': False,
  'committeeTypes': ['Conflict 

In [350]:
status = [
    {
        'isTerminated': item.get('isTerminated', {}),
        # Use the last item of 'statusItemList' if it's not empty
        'status': item.get('statusItemList', None)[-1]['status'] if item.get('statusItemList', None) else None,
        'filerNid': item['filerNid'],
        'Filer Name':item['filerName'],
        'Filer Type': item['committeeTypes']
    } 
    for item in filers_response
]
status_df = pd.DataFrame(status)
# status_df

In [351]:
# get the disclosure addresses
addresses=[{'addressList':item.get('addressList',{}),'filerNid':item['filerNid']} for item in filers_response]
address_dic = {'city':[],'state':[],'zip':[],'filerNid':[]}
for item in addresses:
    for address in item['addressList']:
        if 'Disclosure' in address['addressTypes']:
            address_dic['city'].append(address['city'])
            address_dic['state'].append(address['state'])
            address_dic['zip'].append(address['zip'])
            address_dic['filerNid'].append(item['filerNid'])
address_df=pd.DataFrame(address_dic)
# address_df

In [352]:
status_address_df = status_df.merge(address_df,how='left', on='filerNid')
status_address_df

,isTerminated,status,filerNid,Filer Name,Filer Type,city,state,zip
0,False,ACTIVE,211219898,"Bartsch, Justin",[Conflict of Interest],Oakland,CA,94621
1,False,ACTIVE,211217137,"Hoffman, Kelly",[Conflict of Interest],Oakland,CA,94621
2,False,ACTIVE,211201541,"Gotthelf, Felicia",[Conflict of Interest],Oakland,CA,94612
3,False,ACTIVE,211173329,"Latigue, Candace",[Conflict of Interest],Oakland,CA,94621
4,False,ACTIVE,178429746,"McBride, Alexandria",[Conflict of Interest],Oakland,CA,94610
...,...,...,...,...,...,...,...,...
1268,True,TERMINATED,121706546,McCullough For City Council,[Candidate or Officeholder],Oakland,CA,94609
1269,True,TERMINATED,121706516,Jane Brunner For City Council 2008,[Candidate or Officeholder],Oakland,CA,94609
1270,True,TERMINATED,121706486,Neighbors For Russo,[Candidate or Officeholder],Oakland,CA,94618
1271,True,TERMINATED,121706456,Kaplan For Oakland,[Candidate or Officeholder],Oakland,CA,94612


In [353]:
regs=[{'fppc_id':item.get('registrations',{}).get('CA SOS',None),'filerNid':item['filerNid']} for item in filers_response]
officers=[[item['officers'], item['filerNid']] for item in filers_response if item['officers']]
treasurer_dic={}
for officer in officers:
    if officer[0][0]['position']=='Treasurer':
        treasurer_dic[officer[1]]=officer[0][0]['officerName']
treasurer_dic
for key, value in treasurer_dic.items():
    for item in regs:
        if key==item['filerNid']:
            item['Treasurer']=value
fppc_with_treasurer=[reg for reg in regs if reg.get('Treasurer', None)]
treasurer_df=pd.DataFrame(fppc_with_treasurer)
# treasurer_df

In [354]:
elections=list_elections()
elections

[{'electionNid': '210326313',
  'aid': 'COAK',
  'electionDate': '2026-11-03',
  'electionCaption': '11/3/2026 - General',
  'isPublic': True,
  'electionCodes': 'General',
  'measures': [],
  'seats': [{'seatNid': '210326317',
    'electionNid': '210326313',
    'seatOfficeNid': '121709679',
    'seatCaption': 'City Auditor - City of Oakland',
    'officeName': 'City Auditor - City of Oakland'},
   {'seatNid': '210326329',
    'electionNid': '210326313',
    'seatOfficeNid': '121709698',
    'seatCaption': 'City Council - City of Oakland - 2',
    'officeName': 'City Council - City of Oakland - 2'},
   {'seatNid': '210326366',
    'electionNid': '210326313',
    'seatOfficeNid': '121709717',
    'seatCaption': 'City Council - City of Oakland - 4',
    'officeName': 'City Council - City of Oakland - 4'},
   {'seatNid': '210326368',
    'electionNid': '210326313',
    'seatOfficeNid': '121709771',
    'seatCaption': 'City Council - City of Oakland - 6',
    'officeName': 'City Council -

In [355]:
# get ids from 410s
form410s=get_form('FPPC410')
form410s=form410s[0]
form410={'filerNid':[],'fppc_id':[]}
for form in form410s:
    form410['filerNid'].append(form['filerMeta']['filerId'])
    form410['fppc_id'].append(form.get('filerMeta',{}).get('strings',{}).get('Registration_CA SOS',None))

In [356]:
# get ids from 501s
form501s=get_form('FPPC501')
form501s=form501s[0]
for form in form501s:
    form410['filerNid'].append(form['filerMeta']['filerId'])
    form410['fppc_id'].append(form.get('filerMeta',{}).get('strings',{}).get('Registration_CA SOS',None))

In [357]:
for item in filers_response:
    form410['filerNid'].append(item.get('filerNid',{}))
    form410['fppc_id'].append(item.get('registrations',{}).get('CA SOS',None))
df_410=pd.DataFrame(form410)
df_410.drop_duplicates(inplace=True)
# adding rows with duplicate filerNids with preferential fppc_ids then dropping duplicates keeping last duplicate which should be the one with an id
t2=df_410.dropna()
t1=t2[t2['fppc_id'] != 'Pending']
t1
duped=pd.concat([df_410, t2, t1], ignore_index=True)
best=duped.drop_duplicates(subset=['filerNid'],keep='last',inplace=False)
# best

In [358]:
election_list=[]
previous_df=pd.DataFrame()
for election in elections:
    candidates=election['candidates']
    seats=election['seats']
    election_name=election['electionCaption']
    electionNid=election['electionNid']
    election_year=election['electionDate'][:4]
    election_key={'election_name':election_name, 'electionNid':electionNid, 'election year':election_year}
    election_list.append(election_key)
    if candidates and seats:
        seat_df=pd.DataFrame(seats)
        candidate_df=pd.DataFrame(candidates)
        merge_df=candidate_df.merge(seat_df, on='seatNid')
        current_df=merge_df[['candidateNid','candidateName','seatNid','officeName','electionNid','isIncumbent','isWinner']]
        previous_df=pd.concat([previous_df,current_df],ignore_index=True)
    #merge_df=merge_df.merge(election_key, on='electionNid')
election_df=pd.DataFrame(election_list)
final_df=previous_df.merge(election_df, on='electionNid')
final_df.tail()

,candidateNid,candidateName,seatNid,officeName,electionNid,isIncumbent,isWinner,election_name,election year
361,121707619,"Kernighan, Patricia",149465894,City Council - City of Oakland - 2,125693758,False,False,6/6/2006 - Special,2006
362,133250767,"Drake, Pamela",133250723,City Council - City of Oakland - 2,133250667,False,False,5/17/2005 - Special,2005
363,121710764,"Kakishiba, David",133250723,City Council - City of Oakland - 2,133250667,False,False,5/17/2005 - Special,2005
364,121707619,"Kernighan, Patricia",133250723,City Council - City of Oakland - 2,133250667,False,False,5/17/2005 - Special,2005
365,121710764,"Kakishiba, David",149888803,Director - Oakland Unified School District - 2,149888771,True,False,11/24/2002 - General,2002


In [368]:
final_df

,candidateNid,candidateName,seatNid,officeName,electionNid,isIncumbent,isWinner,election_name,election year
0,210517020,"Afolabi, Baba",207972552,City Council - City of Oakland - 3,200879019,False,False,11/5/2024 - General,2024
1,208539738,"Dunbar, Donnel C",207972552,City Council - City of Oakland - 3,200879019,False,False,11/5/2024 - General,2024
2,161700164,"Fife, Carroll",207972552,City Council - City of Oakland - 3,200879019,False,False,11/5/2024 - General,2024
3,208555055,"Hirsch, Shan M",207972552,City Council - City of Oakland - 3,200879019,False,False,11/5/2024 - General,2024
4,210906799,"Ikharo, Tariq",207972552,City Council - City of Oakland - 3,200879019,False,False,11/5/2024 - General,2024
...,...,...,...,...,...,...,...,...,...
361,121707619,"Kernighan, Patricia",149465894,City Council - City of Oakland - 2,125693758,False,False,6/6/2006 - Special,2006
362,133250767,"Drake, Pamela",133250723,City Council - City of Oakland - 2,133250667,False,False,5/17/2005 - Special,2005
363,121710764,"Kakishiba, David",133250723,City Council - City of Oakland - 2,133250667,False,False,5/17/2005 - Special,2005
364,121707619,"Kernighan, Patricia",133250723,City Council - City of Oakland - 2,133250667,False,False,5/17/2005 - Special,2005


In [360]:
election_ids=list(set(final_df['electionNid'].to_list()))
previous_df=pd.DataFrame()
for id in election_ids:
    influences=list_elections_influences(id)
    influences_dic={'filerNid': [],'electionNid': [],'seatNid': [],'candidateNid': [],'committeeName':[],'election_name': []}
    for candidate in influences:   
        influences_dic['filerNid'].append(candidate.get('filerNid', 'None'))
        influences_dic['election_name'].append(candidate.get('electionCaption', 'None'))
        influences_dic['committeeName'].append(candidate.get('committeeName', 'None'))
        influences_dic['electionNid'].append(candidate.get('electionNid', 'None'))
        influences_dic['seatNid'].append(candidate.get('seatNid', 'None'))
        influences_dic['candidateNid'].append(candidate.get('candidateNid', 'None'))
        current_df=pd.DataFrame(influences_dic)
        current_df=current_df
        previous_df=pd.concat([previous_df,current_df],ignore_index=True)

In [362]:
df3=previous_df
# df3

In [363]:
dfNew = final_df.merge(df3,how='left', on=['candidateNid','election_name','electionNid','seatNid'])
dfNew

,candidateNid,candidateName,seatNid,officeName,electionNid,isIncumbent,isWinner,election_name,election year,filerNid,committeeName
0,210517020,"Afolabi, Baba",207972552,City Council - City of Oakland - 3,200879019,False,False,11/5/2024 - General,2024,210517029,Baba Afolabi for Oakland City Council 2024
1,210517020,"Afolabi, Baba",207972552,City Council - City of Oakland - 3,200879019,False,False,11/5/2024 - General,2024,210517029,Baba Afolabi for Oakland City Council 2024
2,210517020,"Afolabi, Baba",207972552,City Council - City of Oakland - 3,200879019,False,False,11/5/2024 - General,2024,210517029,Baba Afolabi for Oakland City Council 2024
3,210517020,"Afolabi, Baba",207972552,City Council - City of Oakland - 3,200879019,False,False,11/5/2024 - General,2024,210517029,Baba Afolabi for Oakland City Council 2024
4,210517020,"Afolabi, Baba",207972552,City Council - City of Oakland - 3,200879019,False,False,11/5/2024 - General,2024,210517029,Baba Afolabi for Oakland City Council 2024
...,...,...,...,...,...,...,...,...,...,...,...
3480,121707619,"Kernighan, Patricia",149465894,City Council - City of Oakland - 2,125693758,False,False,6/6/2006 - Special,2006,NaN,NaN
3481,133250767,"Drake, Pamela",133250723,City Council - City of Oakland - 2,133250667,False,False,5/17/2005 - Special,2005,NaN,NaN
3482,121710764,"Kakishiba, David",133250723,City Council - City of Oakland - 2,133250667,False,False,5/17/2005 - Special,2005,NaN,NaN
3483,121707619,"Kernighan, Patricia",133250723,City Council - City of Oakland - 2,133250667,False,False,5/17/2005 - Special,2005,NaN,NaN


In [369]:
core_df=dfNew[['candidateName','officeName','committeeName','election_name','filerNid','election year']] 
df4=core_df.merge(best, how='left',on=['filerNid']).drop_duplicates(ignore_index=True)

In [370]:
df5 = df4.merge(treasurer_df, how='left',on=['fppc_id','filerNid'])
df6 = df5.merge(status_address_df, how='left',on=['filerNid'])
df6.columns = ['Candidate Name','officeName', 'committeeName', 'election_name', 'Filler Nid', 'election year', 'FPPC ID', 'treasurer', 'Is Terminated', 'Status', 'Filer Name', 'Filer Type', 'City', 'State', 'Zip']
df6

,Candidate Name,officeName,committeeName,election_name,Filler Nid,election year,FPPC ID,treasurer,Is Terminated,Status,Filer Name,Filer Type,City,State,Zip
0,"Afolabi, Baba",City Council - City of Oakland - 3,Baba Afolabi for Oakland City Council 2024,11/5/2024 - General,210517029,2024,1467002,NaN,False,ACTIVE,Baba Afolabi for Oakland City Council 2024,[Candidate or Officeholder],San Jose,CA,95113
1,"Dunbar, Donnel C",City Council - City of Oakland - 3,"Dunbar, Donnel C",11/5/2024 - General,208539747,2024,None,NaN,False,ACTIVE,"Dunbar, Donnel C",[Person],Oakland,CA,94607
2,"Fife, Carroll",City Council - City of Oakland - 3,Carroll Fife for City Council 2024,11/5/2024 - General,208660692,2024,1463441,"Fife, Carroll",False,ACTIVE,Carroll Fife for City Council 2024,[Candidate or Officeholder],Lakeport,CA,95453
3,"Hirsch, Shan M",City Council - City of Oakland - 3,Committee to Elect Shan Hirsch for City Council,11/5/2024 - General,208555064,2024,1464290,NaN,False,ACTIVE,Committee to Elect Shan Hirsch for City Council,[Candidate or Officeholder],Oakland,CA,94607
4,"Ikharo, Tariq",City Council - City of Oakland - 3,"Ikharo, Tariq",11/5/2024 - General,210906813,2024,None,NaN,False,ACTIVE,"Ikharo, Tariq",[Person],Oakland,CA,94612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,"Kernighan, Patricia",City Council - City of Oakland - 2,NaN,6/6/2006 - Special,NaN,2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
359,"Drake, Pamela",City Council - City of Oakland - 2,NaN,5/17/2005 - Special,NaN,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
360,"Kakishiba, David",City Council - City of Oakland - 2,NaN,5/17/2005 - Special,NaN,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
361,"Kernighan, Patricia",City Council - City of Oakland - 2,NaN,5/17/2005 - Special,NaN,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [371]:
df6.to_csv('output/output.csv')